In [1]:
import cv2
import mediapipe as mp
import numpy as np

In [2]:
# Initialize MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(min_detection_confidence=0.5, min_tracking_confidence=0.5)

I0000 00:00:1723212664.004498       1 gl_context.cc:344] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M1
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [3]:
# Drawing specifications
mp_drawing = mp.solutions.drawing_utils
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)

In [4]:
# Indices for mouth landmarks
MOUTH_LANDMARKS = {
    "inner_upper_lip": [13, 14],
    "inner_lower_lip": [17, 18],
    "left_corner": 61,
    "right_corner": 291
}

In [5]:
def calculate_mouth_aspect_ratio(landmarks):
    # Calculate the vertical distance (height of the mouth)
    inner_upper_lip = np.mean(landmarks[MOUTH_LANDMARKS["inner_upper_lip"]], axis=0)
    inner_lower_lip = np.mean(landmarks[MOUTH_LANDMARKS["inner_lower_lip"]], axis=0)
    vertical_dist = np.linalg.norm(inner_upper_lip - inner_lower_lip)

    # Calculate the horizontal distance (width of the mouth)
    left_corner = landmarks[MOUTH_LANDMARKS["left_corner"]]
    right_corner = landmarks[MOUTH_LANDMARKS["right_corner"]]
    horizontal_dist = np.linalg.norm(left_corner - right_corner)

    return vertical_dist / horizontal_dist

In [13]:
# Video capture
cap = cv2.VideoCapture(0)

yawn_threshold = 0.6  # Adjust this threshold based on observations
yawn_counter = 0
no_of_yawn = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert frame to RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(rgb_frame)

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            # Draw the face mesh landmarks
            #mp_drawing.draw_landmarks(
                #image=frame,
                #landmark_list=face_landmarks,
                #connections=mp_face_mesh.FACEMESH_TESSELATION,
                #landmark_drawing_spec=drawing_spec,
                #connection_drawing_spec=drawing_spec)
            
            mp_drawing.draw_landmarks(
                image=frame,
                landmark_list=face_landmarks,
                connections=mp_face_mesh.FACEMESH_CONTOURS,#mp_face_mesh.FACEMESH_TESSELATION,
                landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=1, circle_radius=1),
                connection_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=1)
            )
            # Extract the landmarks corresponding to the mouth
            landmarks = np.array([(lm.x, lm.y) for lm in face_landmarks.landmark])

            # Calculate mouth aspect ratio (MAR)
            mar = calculate_mouth_aspect_ratio(landmarks)
            print(mar)

            # Detect yawn based on threshold
            if mar > yawn_threshold:
                print('in yawn')
                yawn_counter += 1
                cv2.putText(frame, 'Yawning...', (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
            else:
                yawn_counter = 0

            if yawn_counter > 15:  # Consistent yawn for a few frames
                if yawn_counter == 16:
                    no_of_yawn = no_of_yawn + 1
            cv2.putText(frame, str(no_of_yawn) + ' Yawn Detected', (30, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 255), 2, cv2.LINE_AA)    

    # Display the frame
    cv2.imshow('Yawn Detection', frame)

    if cv2.waitKey(5) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()

0.43448506659509556
0.46871475311551714
0.4487404647096474
0.48063366112343786
0.4568810704836873
0.44859943029745625
0.4455808460259524
0.4492173559643865
0.4522065776834558
0.4461095898039866
0.44665490170534616
0.450893381209307
0.44916419366798793
0.45194072492705756
0.46425211472417627
0.4664238651573494
0.45093527541990813
0.44807797637327845
0.45391928147514204
0.4534899618741624
0.45092306502612645
0.46231071701071513
0.46358471123533906
0.4615818774896764
0.4564643462113238
0.465598164233815
0.46059148912313025
0.4761688428894669
0.4678714107249662
0.4558585168615401
0.4752315090708715
0.4812002518725738
0.46380106549750827
0.4748893054493009
0.46621134340228415
0.4638440906388292
0.4680245795595405
0.4683584095042894
0.46994918241363254
0.4728268694922811
0.4626545209315899
0.4682918486231957
0.4583503713028764
0.4643093687177799
0.46129957032774044
0.4631473261895842
0.4628386909055521
0.46591688580018886
0.46575150307978536
0.46024952172494693
0.4657213007296498
0.463629771

: 